# Instalación de dependencias


In [ ]:
!pipenv install git+https://github.com/joackob/iol-client.git#egg=iol_client

# Configuración de cuenta de usuario


In [ ]:
import iol.src.api.client as client

Se recomienda usar el archivo `.env` para colocar claves privadas, pero este espacio tambien puede ser usado como punto de entrada de datos privados. Tener la debida precaución.


In [ ]:
user = os.getenv("IOL_USER") or ""
password = os.getenv("IOL_PASS") or ""

In [ ]:
client_iol = client.IOLClient(username=user, password=password)

# Historico anual de un determinado instrumento


In [ ]:
from datetime import date
from iol.src.api.constants import Ajustada, Mercado

In [ ]:
hoy = date.today()
hace_un_anio = hoy.replace(year=hoy.year - 1)
historico = await client_iol.get_titulo_historicos(
    simbolo="GGAL",
    mercado=Mercado.BCBA,
    fecha_desde=hace_un_anio,
    fecha_hasta=hoy,
    ajustada=Ajustada.AJUSTADA,
)

In [ ]:
import pandas as pd

df = pd.DataFrame(data=historico)
df.head()

## Guardando los datos


Exportamos datos a un excel


In [ ]:
# df.to_excel("historico.xlsx")

Leer datos desde excel


In [ ]:
# df = pd.read_excel("historico.xlsx")

## Analisis y visualización


Primera vista de los datos


In [ ]:
df.head()

Cantidad de registros encontrados


In [ ]:
df.shape

Graficando variables en función del tiempo


In [ ]:
df.set_index("fechaHora", inplace=True)
df.head()

In [ ]:
df.sort_index(inplace=True)

In [ ]:
fig = df.plot.line(y="ultimoPrecio", rot=0, figsize=(20, 10), grid=True)
fig.set_xlabel("Fecha")
fig.set_ylabel("Precio(pesos argentinos)")

# Intrumentos


In [ ]:
from iol.src.api.constants import Pais


instrumentos = await client_iol.get_instrumentos(pais=Pais.ARG)
instrumentos = pd.DataFrame(data=instrumentos)
instrumentos

# Panel general de un determinado instrumento


In [ ]:
from iol.src.api.constants import Instrumento


panel = await client_iol.get_paneles(pais=Pais.ARG, instrumento=Instrumento.ARG.BONOS)
panel = pd.DataFrame(data=panel)
panel

# Titulos


In [ ]:
titulo = await client_iol.get_titulo(simbolo="CUAP", mercado=Mercado.BCBA)

In [ ]:
titulo = pd.Series(data=titulo)
titulo

# Opciones de un titulo


In [ ]:
opciones_titulo = await client_iol.get_titulo_opciones(
    mercado=Mercado.BCBA, simbolo="GGAL"
)

In [ ]:
opciones_titulo = pd.DataFrame(data=opciones_titulo)

In [ ]:
cotizacion_opciones_titulo = opciones_titulo["cotizacion"].apply(pd.Series)
cotizacion_opciones_titulo = pd.DataFrame(data=cotizacion_opciones_titulo)

In [ ]:
opciones_titulo = opciones_titulo.drop(columns=["cotizacion"])
opciones_titulo.head()

In [ ]:
cotizacion_opciones_titulo.head()

In [ ]:
cotizacion_opciones_titulo.shape, opciones_titulo.shape

# Cotizacion de un titulo


In [ ]:
cotizacion = await client_iol.get_titulo_cotizacion(
    simbolo="GGAL", mercado=Mercado.BCBA
)

In [ ]:
puntas_cotizacion = cotizacion["puntas"]
puntas_cotizacion = pd.DataFrame(data=puntas_cotizacion)

In [ ]:
puntas_cotizacion

In [ ]:
cotizacion = pd.DataFrame(data=cotizacion)
cotizacion = cotizacion.drop(columns=["puntas"])
cotizacion

# Panel de cotizaciones


In [ ]:
from iol.src.api.constants import Panel


panel_cotizaciones = await client_iol.get_panel_cotizaciones(
    pais=Pais.ARG,
    instrumento=Instrumento.ARG.BONOS,
    panel=Panel.ARG.BONOS.SOBERANO_ARS_CER,
)

In [ ]:
titulos_panel_cotizaciones = panel_cotizaciones["titulos"]

In [ ]:
def flat_titulo(titulo):
    return {
        **titulo,
        "puntas": None,
        "precioCompra": titulo["puntas"]["precioCompra"],
        "precioVenta": titulo["puntas"]["precioVenta"],
        "cantidadCompra": titulo["puntas"]["cantidadCompra"],
        "cantidadVenta": titulo["puntas"]["cantidadVenta"],
    }

In [ ]:
titulos_flated = [flat_titulo(titulo) for titulo in titulos_panel_cotizaciones]
titulos_flated = pd.DataFrame(data=titulos_flated)
titulos_flated = titulos_flated.drop(columns=["puntas"])
titulos_flated